## More python to cython

In [1]:
# Change path as needed
MAP_PATH = "../tests/pickle_data/GresvanAIE.xz"

In [2]:
# load cell magic things
%load_ext line_profiler
%load_ext Cython

In [3]:
# imports
import matplotlib.pyplot as plt
import numpy as np

from sc2.ids.unit_typeid import UnitTypeId
from sc2.bot_ai import BotAI
from sc2.position import Point2
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.data import Race
from sc2.unit import Unit
from sc2.units import Units
from tests.load_bot_from_pickle import get_map_specific_bot

In [4]:
# setup a burnysc2 BOTAI instance we can test with
bot: BotAI = get_map_specific_bot(MAP_PATH)

In [ ]:
# sanity check that the bot instance is working
plt.imshow(bot.game_info.pathing_grid.data_numpy)

In [1]:
units = bot.units
unit = units[0]


NameError: name 'bot' is not defined

## `has_creep()`

Default function checks pixel map, this is slower then checking a numpy array.

In [7]:
%timeit bot.has_creep(unit)

1.22 μs ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
%timeit bot.has_creep(unit.position)

1.22 μs ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
def numpy_has_creep(creep_numpy_grid, position: Point2) -> bool:
    return creep_numpy_grid[position[0], position[1]] == 1

In [10]:
%timeit numpy_has_creep(bot.state.creep.data_numpy, unit.position.rounded)

486 ns ± 6.52 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


#### Already 2.5 times faster! Try cython

In [11]:
%%cython
from libc.math cimport floor
cimport numpy as cnp
from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
cpdef bint cython_has_creep(
    cnp.ndarray[cnp.npy_bool, ndim=2] creep_numpy_grid,
    (double, double) position,
):
    """Optimized creep checking function with internal rounding"""
    cdef unsigned int x = int(position[0])
    cdef unsigned int y = int(position[1])
    return creep_numpy_grid[x, y] == 1

In [12]:
%timeit cython_has_creep(bot.state.creep.data_numpy, unit.position)

266 ns ± 2.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
# quick test
creep_copy = bot.state.creep.data_numpy.copy()
creep_copy[5, 10] = 1
cython_has_creep(creep_copy, Point2((5.2, 10.2)))

True

#### Using cython we get ~4.8 times faster over the default burny-sc2 function!

## `in_pathing_grid`

Probably similar to checking `has_creep`

In [14]:
%timeit bot.in_pathing_grid(unit)

1.2 μs ± 8.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [15]:
%timeit bot.in_pathing_grid(unit.position)

1.2 μs ± 5.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [16]:
def numpy_in_pathing_grid(pathing_numpy_grid, position: Point2) -> bool:
    return pathing_numpy_grid[position[0], position[1]] == 1

In [17]:
%timeit numpy_in_pathing_grid(bot.game_info.pathing_grid.data_numpy.T, unit.position.rounded)

629 ns ± 5.28 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [18]:
%%cython
from libc.math cimport floor
cimport numpy as cnp
from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
cpdef bint cython_in_pathing_grid(
    cnp.ndarray[cnp.npy_bool, ndim=2] pathing_numpy_grid,
    (double, double) position,
):
    """Optimized creep checking function with internal rounding"""
    cdef unsigned int x = int(position[0])
    cdef unsigned int y = int(position[1])
    return pathing_numpy_grid[x, y] == 1

In [19]:
%timeit cython_in_pathing_grid(bot.game_info.pathing_grid.data_numpy.T, unit.position)

378 ns ± 3.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [20]:
%%cython
from libc.math cimport floor
cimport numpy as cnp
from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
cpdef bint cython_in_pathing_grid_ma(
    cnp.ndarray[cnp.float32_t, ndim=2] pathing_numpy_grid,
    (double, double) position,
):
    """Optimized creep checking function with internal rounding"""
    cdef unsigned int x = int(position[0])
    cdef unsigned int y = int(position[1])
    return pathing_numpy_grid[x, y] == 1

In [21]:
_grid = bot.game_info.pathing_grid.data_numpy.T.astype(np.float32)

In [22]:
%timeit cython_in_pathing_grid(_grid, unit.position)

ValueError: Buffer dtype mismatch, expected 'npy_bool' but got 'float'

#### Using cython we get ~4.4 times faster over the default burny-sc2 function!

In [ ]:
%%cython
from libc.math cimport floor
cimport numpy as cnp
from cython cimport boundscheck, wraparound
import numpy as np

@boundscheck(False)
@wraparound(False)
cpdef bint cy_point_below_value(
    cnp.ndarray[cnp.npy_float32, ndim=2] grid,
    (double, double) position,
    double weight_safety_limit = 1.0,
):
    """
    987 ns ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
    Python alternative:
    4.66 µs ± 64.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
    """
    cdef double weight = 0.0
    cdef unsigned int x = int(position[0])
    cdef unsigned int y = int(position[1])
    weight = grid[x, y]
    # np.inf check if drone is pathing near a spore crawler
    return weight == np.inf or weight <= weight_safety_limit

In [ ]:
grid = bot.game_info.pathing_grid.data_numpy.T.astype(np.float32)

In [ ]:
%timeit cy_point_below_value(grid, Point2((5, 10)))


In [ ]:
%timeit cy_point_below_value(grid, Point2((5, 10)))

## `cy_closer_than()`


### Python Version:

In [ ]:
%timeit units.closer_than(10.0, unit.position)

In [ ]:
%%cython
from cython cimport boundscheck, wraparound

cpdef double cy_distance_to_squared(
        (double, double) p1,
        (double, double) p2
):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2


@boundscheck(False)
@wraparound(False)
cpdef (list) cy_closer_than(object units, double max_distance, (float, float) position ):
    cdef:
        unsigned int len_units = len(units)
        unsigned int i
        double max_distance_sq = max_distance * max_distance
        double dist_sq
        (float, float) other_pos
        list returned_units = []

    for i in range(len_units):
        other_pos = units[i].position
        dist_sq = cy_distance_to_squared((other_pos[0], other_pos[1]), (position[0], position[1]))
        if dist_sq < max_distance_sq:
            returned_units.append(units[i])

    return returned_units

### Cython Version:

In [ ]:
%timeit cy_closer_than(units, 10.0, unit.position)

### Python Hybrid version of `units.closer_than()` that uses a cython distance calculation.


In [ ]:
from cython_extensions import cy_distance_to_squared, enable_safe_mode
enable_safe_mode(False)
def alternative_hybrid(units, position, distance):
    max_sq = distance * distance
    return [u for u in units
            if cy_distance_to_squared(u.position, position) < max_sq]
    

In [ ]:
%timeit alternative_hybrid(units, unit.position, 10.0)

## `cy_further_than()`

### Python:

In [ ]:
%timeit units.further_than(10.0, unit.position)

In [ ]:
%%cython
from cython cimport boundscheck, wraparound

cpdef double cy_distance_to_squared(
        (double, double) p1,
        (double, double) p2
):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2


@boundscheck(False)
@wraparound(False)

@boundscheck(False)
@wraparound(False)
cpdef (list) cy_further_than(object units, double min_distance, (float, float) position ):
    cdef:
        unsigned int len_units = len(units)
        unsigned int i
        double min_distance_sq = min_distance * min_distance
        double dist_sq
        (float, float) other_pos
        list returned_units = []

    for i in range(len_units):
        other_pos = units[i].position
        dist_sq = cy_distance_to_squared((other_pos[0], other_pos[1]), (position[0], position[1]))
        if dist_sq > min_distance_sq:
            returned_units.append(units[i])

    return returned_units

### Cython:

In [ ]:
%timeit cy_further_than(units, 10.0, unit.position)

## `Cy_structure_pending()`

### Python: Version:

In [12]:
%timeit bot.already_pending(UnitTypeId.SPAWNINGPOOL)

2.06 μs ± 6.49 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Cython Version of `structure_pending()`:

In [5]:
%%cython
from cython cimport boundscheck, wraparound
from cython_extensions.ability_mapping import map_value
@boundscheck(False)
@wraparound(False)
cpdef unsigned int cy_structure_pending(
        object bot,
        object structure_type
    ):
    cdef:
        unsigned int num_pending = 0
        object structures_collection, structure
        object counts_and_progress = bot._abilities_count_and_build_progress[0]

        cdef int count
        cdef int target = <int> structure_type.value


    for ability_obj, count in counts_and_progress.items():
        ability_int = <int> ability_obj.value
        if map_value(target) == ability_int:
            num_pending += count

    return num_pending



In [6]:
%timeit cy_structure_pending(bot, UnitTypeId.SUPPLYDEPOT)

939 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Cython Version of `structure_pending()`: 

In [ ]:
%%cython
from cython cimport boundscheck, wraparound
from cython_extensions.ability_mapping import map_value
from cython_extensions.ability_order_tracker import cy_abilities_count_structures


cdef struct AbilityCount:
    int ability_id
    int count

@boundscheck(False)
@wraparound(False)
cpdef unsigned int cy_structure_pending(
        object bot,
        object structure_type,
    ):
    cdef:
        unsigned int num_pending = 0
        object counts_and_progress
        object creation_ability = None
        object local_get
        int ability_int
        object ability_obj
        int count
        int target = <int> structure_type.value
        cdef AbilityCount item

    # Use optimized Cython function to get ability counts

    counts_and_progress = cy_abilities_count_structures(bot) #returns a c array memoryview


    # local_get reduces repeated attribute accesses
    arr_len = counts_and_progress.shape[0]
    target_created_ability = <int> map_value(target)
    if 0 <= target_created_ability < arr_len:
        item = counts_and_progress[target_created_ability]
        num_pending += item.count
    return num_pending

Content of stdout:
_cython_magic_3e2a234224e4e11aa777758a126603cd3294f15a087708c4eab903c51d563993.c
   Bibliothek "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_3e2a234224e4e11aa777758a126603cd3294f15a087708c4eab903c51d563993.cp313-win_amd64.lib" und Objekt "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_3e2a234224e4e11aa777758a126603cd3294f15a087708c4eab903c51d563993.cp313-win_amd64.exp" werden erstellt.
Code wird generiert.
Codegenerierung ist abgeschlossen.

In [6]:
build_tracker_dict: dict = {
    4354211841: {
        'id': UnitTypeId.NEXUS,
        'target': (59.5, 138.5),
        'time_order_commenced': 77.94642857142857,
        'building_purpose': "DOME_STRING_HERE",  
        'structure_order_complete': True
    }
}


In [8]:
%timeit cy_structure_pending(bot, UnitTypeId.NEXUS)

942 ns ± 3.42 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


##### Hint: You need to pass `include_ares_planned=True` to include Ares planned buildings in the count.
##### This version is around 15% slower than the default cython version without Ares buildings, but still twice as fast as the pure python version.

In [18]:
from collections import Counter
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId
from sc2.constants import (
    ALL_GAS,
    CREATION_ABILITY_FIX,
    IS_PLACEHOLDER,
)
def _abilities_count_and_build_progress(self) -> tuple[Counter[AbilityId], dict[AbilityId, float]]:
        """Cache for the already_pending function, includes protoss units warping in,
        all units in production and all structures, and all morphs"""
        abilities_amount: Counter[AbilityId] = Counter()
        unit: Unit
        for unit in self.units + self.structures:
            for order in unit.orders:
                abilities_amount[order.ability.exact_id] += 1
            if not unit.is_ready and (self.race != Race.Terran or not unit.is_structure):
                # If an SCV is constructing a building, already_pending would count this structure twice
                # (once from the SCV order, and once from "not structure.is_ready")
                if unit.type_id in CREATION_ABILITY_FIX:
                    if unit.type_id == UnitTypeId.ARCHON:
                        # Hotfix for archons in morph state
                        creation_ability = AbilityId.ARCHON_WARP_TARGET
                        abilities_amount[creation_ability] += 2
                    else:
                        # Hotfix for rich geysirs
                        creation_ability = CREATION_ABILITY_FIX[unit.type_id]
                        abilities_amount[creation_ability] += 1
                else:
                    creation_ability: AbilityId = self.game_data.units[unit.type_id.value].creation_ability.exact_id
                    abilities_amount[creation_ability] += 1
        return abilities_amount

In [19]:
%timeit _abilities_count_and_build_progress(bot)

21.5 μs ± 84 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
%%cython
from sc2.data import Race
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId

# Hardcoded fixes for specific unit types that don't have standard creation abilities
cdef dict CREATION_ABILITY_FIX = {
    UnitTypeId.ARCHON: AbilityId.ARCHON_WARP_TARGET,
    # Add other rich geyser or special cases here as needed
}

cpdef dict cy_abilities_count_and_build_progress(object bot):
    """
    Fast Cython implementation to count abilities in progress.
    
    Returns a dict mapping AbilityId -> count (int).
    This replaces bot._abilities_count_and_build_progress[0] for counting.
    
    Args:
        bot: BotAI instance with units, structures, race, and game_data
        
    Returns:
        dict[AbilityId, int]: Counter of ability exact_ids currently active
    """
    cdef:
        dict abilities_amount = {}
        object unit
        object order
        object creation_ability
        Py_ssize_t i, j, len_units, len_orders
        object units_list
        object structures_list
        object all_units
        int current_count
        bint is_terran = bot.race == Race.Terran
        object local_get = abilities_amount.get
        
    # Combine units and structures into a single list once to avoid repeated concatenation
    units_list = bot.units
    structures_list = bot.structures
    
    # Process all units and structures
    len_units = len(units_list)
    for i in range(len_units):
        unit = units_list[i]
        
        # Count orders for this unit
        len_orders = len(unit.orders)
        for j in range(len_orders):
            order = unit.orders[j]
            creation_ability = order.ability.exact_id
            # Fast increment: read current count, add 1, store back
            current_count = <int> local_get(creation_ability, 0)
            abilities_amount[creation_ability] = current_count + 1
        
        # Count units/structures not yet ready (in production/construction)
        # Skip Terran structures (SCV building counts would double-count)
        if not unit.is_ready and (not is_terran or not unit.is_structure):
            # Check if this unit type needs special handling
            if unit.type_id in CREATION_ABILITY_FIX:
                if unit.type_id == UnitTypeId.ARCHON:
                    # Archon morphs from 2 units, count as 2
                    creation_ability = AbilityId.ARCHON_WARP_TARGET
                    current_count = <int> local_get(creation_ability, 0)
                    abilities_amount[creation_ability] = current_count + 2
                else:
                    # Other special cases (e.g., rich geysers)
                    creation_ability = CREATION_ABILITY_FIX[unit.type_id]
                    current_count = <int> local_get(creation_ability, 0)
                    abilities_amount[creation_ability] = current_count + 1
            else:
                # Standard case: lookup creation ability from game data
                try:
                    creation_ability = bot.game_data.units[unit.type_id.value].creation_ability.exact_id
                    current_count = <int> local_get(creation_ability, 0)
                    abilities_amount[creation_ability] = current_count + 1
                except Exception:
                    # Skip units without valid creation ability
                    pass
    
    # Process structures
    len_units = len(structures_list)
    for i in range(len_units):
        unit = structures_list[i]
        
        # Count orders
        len_orders = len(unit.orders)
        for j in range(len_orders):
            order = unit.orders[j]
            creation_ability = order.ability.exact_id
            current_count = <int> local_get(creation_ability, 0)
            abilities_amount[creation_ability] = current_count + 1
        
        # Count structures not yet ready (skip Terran structures being built)
        if not unit.is_ready and (not is_terran or not unit.is_structure):
            if unit.type_id in CREATION_ABILITY_FIX:
                if unit.type_id == UnitTypeId.ARCHON:
                    creation_ability = AbilityId.ARCHON_WARP_TARGET
                    current_count = <int> local_get(creation_ability, 0)
                    abilities_amount[creation_ability] = current_count + 2
                else:
                    creation_ability = CREATION_ABILITY_FIX[unit.type_id]
                    current_count = <int> local_get(creation_ability, 0)
                    abilities_amount[creation_ability] = current_count + 1
            else:
                try:
                    creation_ability = bot.game_data.units[unit.type_id.value].creation_ability.exact_id
                    current_count = <int> local_get(creation_ability, 0)
                    abilities_amount[creation_ability] = current_count + 1
                except Exception:
                    pass
    
    return abilities_amount

Content of stdout:
_cython_magic_435fd3642329c591479b0b4fdcafffa5af1746aeec6322163c7f8801f143238e.c
   Bibliothek "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_435fd3642329c591479b0b4fdcafffa5af1746aeec6322163c7f8801f143238e.cp313-win_amd64.lib" und Objekt "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_435fd3642329c591479b0b4fdcafffa5af1746aeec6322163c7f8801f143238e.cp313-win_amd64.exp" werden erstellt.
Code wird generiert.
Codegenerierung ist abgeschlossen.

In [21]:
%timeit cy_abilities_count_and_build_progress(bot)

16.1 μs ± 59.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
%%cython
from cython cimport boundscheck, wraparound
from cython_extensions.ability_mapping import map_value
from cython_extensions.ability_order_tracker import cy_abilities_count_and_build_progress
@boundscheck(False)
@wraparound(False)
cpdef unsigned int cy_structure_pending_updated(
        object bot,
        object structure_type,
    ):
    cdef:
        unsigned int num_pending = 0
        object counts_and_progress
        object creation_ability = None
        object local_get
        int ability_int
        object ability_obj
        int count
        int target = <int> structure_type.value

    # Use optimized Cython function to get ability counts
    counts_and_progress = cy_abilities_count_and_build_progress(bot)

    # Fast path: resolve creation ability id once and read the precomputed count


    # local_get reduces repeated attribute accesses
    for ability_obj in counts_and_progress.keys():
        ability_int = <int> ability_obj.value
        if ability_int == map_value(target):
            num_pending += <int> counts_and_progress[ability_obj]
    return num_pending

In [6]:
%timeit cy_structure_pending_updated(bot, UnitTypeId.NEXUS)

17.1 μs ± 46.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
%%cython
from sc2.data import Race
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId

from cython_extensions.ability_mapping import map_value
from collections import Counter
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId
from sc2.data import Race
from cython cimport boundscheck, wraparound
from libc.stdlib cimport malloc, free
from libc.string cimport memset

cdef struct AbilityCount:
    int ability_id
    int count

# Disable Python checks for speed
@boundscheck(False)
@wraparound(False)
cpdef AbilityCount[:] abilities_count_structures(object bot):
    """
    Build a C array indexed by ability_id that stores counts.
    Returns: memoryview of AbilityCount (size = 5000)
    """

    cdef int MAX_ABILITIES = 1200
    cdef AbilityCount* arr = <AbilityCount*> malloc(MAX_ABILITIES * sizeof(AbilityCount))
    memset(arr, 0, MAX_ABILITIES * sizeof(AbilityCount))

    cdef object unit
    cdef object order
    cdef int aid

    # Workers orders → ability_id count
    for unit in bot.workers:
        for order in unit.orders:
            aid = <int> order.ability.exact_id.value
            if 0 <= aid < MAX_ABILITIES:
                arr[aid].count += 1

    # Structures → build progress < 1.0 → increment creation ability
    if bot.race != Race.Terran:
        for unit in bot.structures:
            if unit.build_progress < 1.0:
                aid = <int> map_value(unit.type_id.value)
                if 0 <= aid < MAX_ABILITIES:
                    arr[aid].count += 1

    # Return as Python-usable memoryview
    return <AbilityCount[:MAX_ABILITIES]> arr

def cache_per_game_loop(func):
    cache_name = f"_{func.__name__}_cache"
    loop_name = f"_{func.__name__}_loop"
    def wrapper(bot, *args, **kwargs):
        current_loop = bot.state.game_loop
        if getattr(bot, loop_name, None) == current_loop:
            return getattr(bot, cache_name)
        result = func(bot, *args, **kwargs)
        setattr(bot, cache_name, result)
        setattr(bot, loop_name, current_loop)
        return result
    return wrapper

@cache_per_game_loop
def cy_abilities_count_structures(bot):
    return abilities_count_structures(bot)

Content of stdout:
_cython_magic_79061d0c42a12a637d15b98c036870b90b71627f5c4dc3a6a45b01b31e8c9c7e.c
   Bibliothek "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_79061d0c42a12a637d15b98c036870b90b71627f5c4dc3a6a45b01b31e8c9c7e.cp313-win_amd64.lib" und Objekt "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_79061d0c42a12a637d15b98c036870b90b71627f5c4dc3a6a45b01b31e8c9c7e.cp313-win_amd64.exp" werden erstellt.
Code wird generiert.
Codegenerierung ist abgeschlossen.

In [25]:
%timeit cy_abilities_count_structures(bot)

149 ns ± 1.66 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
